In [9]:
import pandas as pd
from scipy.spatial.distance import cosine

In [10]:
import csv
def ReadFile(filename):
    f = open(filename)
    r = csv.reader(f)
    mentions = dict()
    for line in r:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
#         discipline, type_d, sem_rate, bonus, exam_rate = [int(i) for i in line[6:11]]
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [11]:
#data = ReadFile("raw_data32.csv")
#data

In [12]:
# косинусная мера : насколько два пользователя похожи друг на друга. 
def distCosine (vecA, vecB):
    def dotProduct (vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB:
                d += vecA[dim]*vecB[dim]
        return d
    a = dotProduct (vecA,vecB)
    b = math.sqrt(dotProduct(vecA,vecA))
    c = math.sqrt(dotProduct(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [13]:
import math
def makeRecommendation (studentID, studentRates, nBestStudents, nBestDisciplines):
    matches = [(u, distCosine(studentRates[studentID], studentRates[u])) for u in studentRates if u != studentID]
    bestMatches = sorted(matches, key=lambda q :(q[1],q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("  StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    sim = dict()
    sim_all = sum([x[1] for x in bestMatches])
    bestMatches = dict([x for x in bestMatches if x[1] > 0.0])        
    for relatedStudent in bestMatches:
        for discipline in studentRates[relatedStudent]:
            if not discipline in studentRates[studentID]:
                if not discipline in sim:
                    sim[discipline] = 0.0
                sim[discipline] += studentRates[relatedStudent][discipline] * bestMatches[relatedStudent]
    for discipline in sim:
        sim[discipline] /= sim_all
    #bestDisciplines = sorted(sim.iteritems(), key=lambda q :(q[0],q[1]), reverse=True)[:nBestDisciplines]
    bestDisciplines = sorted(sim.items(), key=lambda q :(q[1],q[0]), reverse=True)[:nBestDisciplines]
    print ("Most correlated disciplines:")
    for disciplineInfo in bestDisciplines:    
        print ("  DisciplineID: " + str(disciplineInfo[0]) + " CorrelationCoeff: " + str(disciplineInfo[1]))
    return [(x[0], x[1]) for x in bestDisciplines]

In [14]:
rec = makeRecommendation ('6', ReadFile("raw_data32.csv"), 3,4)

Most correlated with studentID: 6
  StudentID: 7  Coeff: 0.9777545782936696
  StudentID: 9  Coeff: 0.9291546126464784
  StudentID: 5  Coeff: 0.8879638025780343
Most correlated disciplines:
  DisciplineID: 1830 CorrelationCoeff: 31.77116830129253
  DisciplineID: 2004 CorrelationCoeff: 0.0
